In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from nbdev import *
%nbdev_default_export nodes

Cells will be exported to pct.nodes,
unless a different module is specified after an export flag: `%nbdev_export special.module`


# Nodes


In [ ]:
#from pct.rmath import *

In [ ]:
#from pct.functions import Integration

In [ ]:
%nbdev_export
class PCTNode():
    "A single PCT controller."
    def __init__(self, perception):
        self.perceptionCollection = [perception]
        reference = Constant(1)
        self.referenceCollection = [reference]
        comparator = Subtract()
        self.comparatorCollection = [comparator]
        self.outputCollection = [Integrator(75, 100)]
    
    def __call__(self):
        for perceptionFunction in perceptionCollection:
            perceptionFunction()

        for referenceFunction in referenceCollection:
            referenceFunction()               
            
        comparator.addLink(referenceCollection[-1])
        comparator.addLink(perceptionCollection[-1])

        for comparatorFunction in comparatorCollection:
            comparatorFunction()

        outputCollection[0].setLink()
        for outputFunction in outputCollection:
            outputFunction()
            
        self.output = outputCollection[-1]
        return self.output

In [ ]:
#world = Integration(50, 100)
#node = PCTNode(world)
#output = node()

In [ ]:
#hide
from nbdev import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_rmath.ipynb.
Converted 02_functions.ipynb.
Converted 03_nodes.ipynb.
Converted 04_hierarchy.ipynb.
Converted index.ipynb.
